<a href="https://colab.research.google.com/github/majdiskr/models/blob/master/transfer_yolov5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/SShowbiz/yolov5-transfer-learning.git
!pip install -U -r yolov5-transfer-learning/requirements.txt  # install dependencies
!git reset --hard 886f1c03d839575afecb059accf74296fad395b6
%cd /content/yolov5-transfer-learning

Cloning into 'yolov5-transfer-learning'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 118 (delta 24), reused 116 (delta 22), pack-reused 0
Receiving objects: 100% (118/118), 866.33 KiB | 19.25 MiB/s, done.
Resolving deltas: 100% (24/24), done.
     |████████████████████████████████| 10.3 MB 13.0 MB/s 
     |████████████████████████████████| 15.7 MB 183 kB/s 
     |████████████████████████████████| 49.9 MB 14 kB/s 
     |████████████████████████████████| 3.0 MB 42.2 MB/s 
     |████████████████████████████████| 636 kB 47.4 MB/s 
     |████████████████████████████████| 28.5 MB 43 kB/s 
     |████████████████████████████████| 76 kB 5.1 MB/s 
     |████████████████████████████████| 292 kB 50.8 MB/s 
     |████████████████████████████████| 11.3 MB 40.3 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully u

fatal: not a git repository (or any of the parent directories): .git
/content/yolov5-transfer-learning


In [2]:
import torch
from IPython.display import Image  # for displaying images
from utils.google_utils import gdrive_download  # for downloading models/datasets
print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

torch 1.9.0+cu102 _CudaDeviceProperties(name='Tesla K80', major=3, minor=7, total_memory=11441MB, multi_processor_count=13)


[link text](https://)# Download Correctly Formatted Custom Dataset 


In [3]:

%cd /content
!curl -L "https://app.roboflow.com/ds/jqEpQ2UwRT?key=8cWtkOUh6B" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   896  100   896    0     0    314      0  0:00:02  0:00:02 --:--:--   314
100 19.6M  100 19.6M    0     0  5314k      0  0:00:03  0:00:03 --:--:--  192M
Archive:  roboflow.zip
 extracting: README.roboflow.txt     
 extracting: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/130_jpg.rf.1e9f389e9924d169de1dca2bf040866e.jpg  
 extracting: test/images/131_jpg.rf.8ea56743c5b8596b960dbf0e009a42f7.jpg  
 extracting: test/images/153_jpg.rf.a364d1f6f7eed731e274a5ae0ebe0994.jpg  
 extracting: test/images/501_jpg.rf.7a2b93f33bac5faefebe5be793fcfe64.jpg  
 extracting: test/images/701_jpg.rf.ee732a9e82fe0c9773ce0070d8c1062e.jpg  
 extracting: test/images/711_jpg.rf.3a35cfa3710eb3a5d7a60b5af0255207.jpg  
 extracting: test/images/FudanPed00014_png.rf.67b276dd69304d60ece5aec97cb54a44.jp

# Define Model Configuration and Architecture


In [4]:
# define number of classes based on YAML
%cd /content
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

/content


In [5]:
#this is the model configuration we will use
%cat /content/yolov5-transfer-learning/models/yolov5s.yaml

# parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C

In [11]:
%cd /content/yolov5-transfer-learning
!python model.info()

/content/yolov5-transfer-learning
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `python model.info()'


In [15]:
#$ train.py --batch 48 --weights yolov5s.pt --data voc.yaml --epochs 50 --cache --img 512 --hyp hyp.finetune.yaml
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/yolov5-transfer-learning
!python train.py  --img 416 --batch 16 --epochs 200 --data '../data.yaml' --cfg ./models/yolov5s.yaml --weights 'yolov5s.pt' --name yolov5s_results  --cache


/content/yolov5-transfer-learning
github: up to date with https://github.com/SShowbiz/yolov5-transfer-learnin ✅
YOLOv5 🚀 bb9cd04 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=16, bbox_interval=-1, bucket='', cache_images=True, cfg='./models/yolov5s.yaml', data='../data.yaml', device='', entity=None, epochs=200, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[416, 416], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='yolov5s_results', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/yolov5s_results2', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=16, upload_dataset=False, weights='yolov5s.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.

# Evaluate Custom YOLOv5 Detector Performance

In [8]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

Launching TensorBoard...

KeyboardInterrupt: ignored

In [ ]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason... 
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/content/yolov5-transfer-learning/runs/train/yolov5s_results/results.png', width=1000)  # view results.png

### Curious? Visualize Our Training Data with Labels

After training starts, view `train*.jpg` images to see training images, labels and augmentation effects.

Note a mosaic dataloader is used for training (shown below), a new dataloading concept developed by Glenn Jocher and first featured in [YOLOv4](https://arxiv.org/abs/2004.10934).

In [ ]:
# first, display our ground truth data
print("GROUND TRUTH TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results/test_batch0_labels.jpg', width=900)

In [ ]:
# print out an augmented training example
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results/train_batch0.jpg', width=900)